In [55]:
import cv2
from scipy.spatial import distance as dist
import numpy as np
import time
import dlib
from collections import OrderedDict
from imutils import face_utils

In [56]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    
    #compute ear
    ear = (A+B)/(2.0*C)
    
    return ear

In [1]:
#SPECIFICALLY TO CONVERT DLIB OBJ. TO NUMPY ARRAY
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
 
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
 
    # return the list of (x, y)-coordinates
    return coords

In [58]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3

# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

In [59]:
facial_features = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])

In [60]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/tejas/Desktop/CV/shape_predictor_68_face_landmarks.dat')

In [61]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lstart, lend) = facial_features['left_eye']
(rstart,rend) = facial_features['right_eye']        

In [70]:
cap = cv2.VideoCapture(0)


while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detect faces in the greyscale frame
    rects = detector(gray,0)
    
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray,rect)
        shape = shape_to_np(shape)
        
        #extract the left and right eye co-ords. to compute ear
        leye = shape[lstart:lend]
        reye = shape[rstart:rend]
        lEAR = eye_aspect_ratio(leye)
        rEAR = eye_aspect_ratio(reye)
        
        #avg. the EAR together for both the eyes
        ear = (lEAR + rEAR) / 2.0
        
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leye_hull = cv2.convexHull(leye)
        reye_hull = cv2.convexHull(reye)
        cv2.drawContours(frame, [leye_hull], -1, (0,255,0), 1)
        cv2.drawContours(frame, [reye_hull], -1, (0,255,0), 1)
        
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
                
            COUNTER = 0
            
        cv2.putText(frame,"Blinks : {}".format(TOTAL), (10,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
        cv2.putText(frame,"EAR : {:.2f}".format(ear), (300,30), 
                    cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    
    #cv2.imshow('gray',gray)
    frame = cv2.resize(frame,(1200,900))
    cv2.imshow('frame',frame)
    #if the key = 1 and q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
TOTAL = 0
COUNTER = 0

2